# Preface
This assignment has a lot of wide matricies. For the best viewing experience, please run the code below to increase the notebook width to 100% of your browser width. This will make the HITS and PAGERANK matricies fully legible with increased precision.

In [100]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# 7.1 Creation of the crawler
  
I first decided to make the crawler items. Because I will have to use it to eventually create a stochastic matrix, I decided to hash all the links to make lookups fast and easy. I also decided to leave in the non-hashed version of the links to keep the data legable. The `num_of_links` makes the weighting computation easier.
  
  
Also because scrapy hates me a refuses to work when I use their own generated spider templates, I am creating ^a custom ~~classes~~ ^dictionary template to hold my data and using the scrapy shell commands to populate them. Looks like its time to learn JSON.

In [1]:
#class paradoxItem(object):
#    def __init__(self, base_link=None, outward_links=None,
#                 num_of_links=None, base_link_hash=None, 
#                 outward_links_hashes=None):
#        self.base_link = base_link
#        self.outward_links = outward_links
#        self.num_of_links = num_of_links
#        self.base_link_hash = base_link_hash
#        self.outward_links_hashes = outward_links_hashes

def dict_format(base_link=None, outward_links=None,
                num_of_links=None, base_link_hash=None, 
                outward_links_hashes=None):
    return {
                'base_link' : base_link,
                'outward_links' : outward_links,
                'num_of_links' : num_of_links,
                'base_link_hash' : base_link_hash,
                'outward_links_hashes' : outward_links_hashes
        
            }

~~Class~~ Dictionary template created, lets create a ^master dictionary to store them. (More on why a dictionary later)

In [2]:
pItems = {}

Now lets choose a site and start experimenting with the scrapy shell commands to gather the data.

In [3]:
import requests
import re
from scrapy.http import TextResponse

#running scrappy to get a response from a URL
r = requests.get('https://www.paradoxplaza.com/')
response = TextResponse(r.url, body=r.text, encoding='utf-8')

In [4]:
for link in range(20):
    print response.xpath('//li/a/@href')[link].extract()

https://www.paradoxplaza.com/account
https://www.paradoxplaza.com/register
https://www.paradoxplaza.com/
https://www.paradoxplaza.com/account
https://www.paradoxplaza.com/register
https://www.paradoxplaza.com/browse/
https://www.paradoxplaza.com/new-releases/
https://www.paradoxplaza.com/coming-soon/
https://www.paradoxplaza.com/on-sale/
https://play.paradoxplaza.com
https://forum.paradoxplaza.com/forum/index.php
http://www.paradoxwikis.com
https://www.paradoxplaza.com/support-static-info-ca.html
http://merch.paradoxplaza.com/
https://www.paradoxplaza.com/news
https://www.paradoxplaza.com/about-us-static-info-ca.html
http://career.paradoxplaza.com/
https://www.paradoxplaza.com/playtest-static-info-ca.html
https://www.paradoxplaza.com/press-static-info-ca.html
https://www.facebook.com/ParadoxInteractive


And there is 20 links right there. 

### EDIT: DETOUR 1, Validating URLs
Further along, I realised that as I branched out more and more I eventually start getting links that are not valid urls. Like so...

In [5]:
for link in range(20):
    print response.xpath('//a/@href')[link].extract()

/
#
https://www.paradoxplaza.com/games/
http://forum.paradoxplaza.com/
https://play.paradoxplaza.com/
https://www.paradoxplaza.com/on-sale/
http://merch.paradoxplaza.com/
https://www.paradoxplaza.com/account?fromLoginUrl=true
https://www.paradoxplaza.com/account
https://www.paradoxplaza.com/register
/browse/
https://www.paradoxplaza.com/
https://www.paradoxplaza.com/account
https://www.paradoxplaza.com/register
https://www.paradoxplaza.com/manage-cookies-page.html
https://www.paradoxplaza.com/surviving-mars-space-race/SUSM01ESK0000063-MASTER.html
/battletech---flashpoint/BTBT01ESK0000001-MASTER.html
/battletech---flashpoint/BTBT01ESK0000001-MASTER.html
/battletech---flashpoint/BTBT01ESK0000001-MASTER.html
/battletech---flashpoint/BTBT01ESK0000001-MASTER.html


Using `xpath('//li/a/@href')` instead of `xpath('//a/@href')` was a quick fix to filter out the garbage links on the homepage, but has comback to bite me in the ass with my `parse_links_beta()`[SPOILERS]. Eventually I run into some pages where some links are mearly url-extensions.
  
So lets spend some time fixing that...

In [6]:
num_of_valid_links = 0
for link in range(20):
    if(response.xpath('//a/@href')[link].extract().startswith('http')):
        print response.xpath('//a/@href')[link].extract()
        num_of_valid_links += 1
        
print "Number of Valid Links: "+str(num_of_valid_links)

https://www.paradoxplaza.com/games/
http://forum.paradoxplaza.com/
https://play.paradoxplaza.com/
https://www.paradoxplaza.com/on-sale/
http://merch.paradoxplaza.com/
https://www.paradoxplaza.com/account?fromLoginUrl=true
https://www.paradoxplaza.com/account
https://www.paradoxplaza.com/register
https://www.paradoxplaza.com/
https://www.paradoxplaza.com/account
https://www.paradoxplaza.com/register
https://www.paradoxplaza.com/manage-cookies-page.html
https://www.paradoxplaza.com/surviving-mars-space-race/SUSM01ESK0000063-MASTER.html
Number of Valid Links: 13


And done. One thing to note about this fix, it merely throws out bad links, but will still only search the first 20. This is why only 13 links are displayed

### END DETOUR 1

Lets experiment with pulling links from within links, setting a hard cap at ~~200 links,~~ 15 links per page with a max depth of 3.

EDIT: for some reason my link counter was throwing off the recursion. Instead I will use the depth and links per page to determine the max links requested. 
  
`Max_Links_Requested = links_per_page^(depth)`
  
Now this isnt the actual number of links I recieve, because all url-extensions and `'paradoxwiki'` links are discarded.
  
`Number_of_Links = links_per_page^(depth-1) - invalid_links`

In [7]:
def min(a,b):
    if(a < b):
        return a
    else:
        return b

In [8]:
#MAX_LINKS = 200
MAX_PER_PAGE = 20
MAX_DEPTH = 3
num_of_valid_links = 0
final_print = False

def parse_links_alpha(url,depth):
    global num_of_valid_links
    global final_print
    
    if(depth == 3):
        return;
#    elif(num_of_links > MAX_LINKS):
#        if(final_print == False):
#            print "Link Limit Reached"
#            final_print = True
#        return
    else:
        print "\t"*depth+url
    
    r = requests.get(url)
    response = TextResponse(r.url, body=r.text, encoding='utf-8')
    
    num_of_links_on_page = len(response.xpath('//li/a/@href').extract())
    
    # sort through either the max number allowed, or the number of links avalible
    for link in range(min(MAX_PER_PAGE,num_of_links_on_page)):
        
        # if the response starts with 'http' its not a short link and thus valid
        # any 'https://paradoxwikis.com/' requests automatically crash the program
        #  so all of them are to be ignored
        if(response.xpath('//li/a/@href')[link].extract().startswith('http') and
           'paradoxwikis' not in response.xpath('//li/a/@href')[link].extract()):
            num_of_valid_links += 1
            parse_links_alpha(response.xpath('//li/a/@href')[link].extract(),depth+1)
    
parse_links_alpha('https://www.paradoxplaza.com/',0)
print "Number of Valid Links: "+str(num_of_valid_links)

https://www.paradoxplaza.com/
	https://www.paradoxplaza.com/account
		https://www.paradoxplaza.com/account
		https://www.paradoxplaza.com/register
		https://www.paradoxplaza.com/login?original=%2Faccount
		https://www.paradoxplaza.com/account
		https://www.paradoxplaza.com/register
		https://www.paradoxplaza.com/browse/
		https://www.paradoxplaza.com/new-releases/
		https://www.paradoxplaza.com/coming-soon/
		https://www.paradoxplaza.com/on-sale/
		https://play.paradoxplaza.com
		https://forum.paradoxplaza.com/forum/index.php
		https://www.paradoxplaza.com/support-static-info-ca.html
		http://merch.paradoxplaza.com/
		https://www.paradoxplaza.com/news
		https://www.paradoxplaza.com/about-us-static-info-ca.html
		http://career.paradoxplaza.com/
		https://www.paradoxplaza.com/playtest-static-info-ca.html
		https://www.paradoxplaza.com/press-static-info-ca.html
		https://www.facebook.com/ParadoxInteractive
	https://www.paradoxplaza.com/register
		https://www.paradoxplaza.com/account
		htt

		https://www.paradoxplaza.com/coming-soon/?prefn1=pdx-type&prefv1=dlc
		https://www.paradoxplaza.com/coming-soon/?prefn1=pdx-type&prefv1=expansion
		https://www.paradoxplaza.com/coming-soon/?prefn1=pdx-type&prefv1=edition
		https://www.paradoxplaza.com/coming-soon/?prefn1=pdx-genres&prefv1=Grand%20Strategy
		https://www.paradoxplaza.com/coming-soon/?prefn1=pdx-genres&prefv1=Historical
	https://www.paradoxplaza.com/on-sale/
		https://www.paradoxplaza.com/account
		https://www.paradoxplaza.com/register
		https://www.paradoxplaza.com/on-sale/
		https://www.paradoxplaza.com/account
		https://www.paradoxplaza.com/register
		https://www.paradoxplaza.com/on-sale/?sz=16&start=16
		https://www.paradoxplaza.com/on-sale/?sz=16&start=32
		https://www.paradoxplaza.com/browse/
		https://www.paradoxplaza.com/new-releases/
		https://www.paradoxplaza.com/coming-soon/
		https://www.paradoxplaza.com/on-sale/
		https://play.paradoxplaza.com
		https://forum.paradoxplaza.com/forum/index.php
		https://www.p

This works, but there is a lot of repetition. Let's utilize our custom ~~class~~ dictionary template to store crawled links. Then we will put them in a dictionary with the key being the base url hashed. This is the primary reason why I wanted to use the dictionary, to make hash lookups much easier. Might as well let the python do the heavy lifting for this one.

*This code takes a while to chug along. I've been using a 3x7 (`MAX_DEPTH` of 3 and `MAX_PER_PAGE` of 20) which gives me a maxmum of 8000 links. In reality I get just under 3000 valid links, which are made up of only **47 unique links!!!** To stay near 200 total links, the proposed limit, use 4x4 or 5x3 which gives you 256 and 243 links respectively. I do not recommend this through because you will only get a handful of unique links that way*

Edit: I changed the `MAX_PER_PAGE` to 15 in the code below because a 47 width matrix is too wide for juypter to display nicely even on full screen. So I downed it to a 3x15 which gives me 525 links of which 13 are unique.

In [9]:
import hashlib

#MAX_LINKS = MAX_PER_PAGE^MAX_DEPTH
MAX_DEPTH = 3
MAX_PER_PAGE = 15
num_of_valid_links = 0
final_print = False

pItems = {}

def parse_links_beta(url,depth):
    global num_of_valid_links
    global pItems
    
    if(depth == MAX_DEPTH):
        return;
    else:
        print "\t"*depth+url
    
    r = requests.get(url)
    response = TextResponse(r.url, body=r.text, encoding='utf-8')
    
    #building dict is non-recursive
    base_link = url
    
    num_of_links_on_page = len(response.xpath('//li/a/@href').extract())
    outward_links = []
    # read either the max number allowed, or the number of links avalible
    for link in range(min(MAX_PER_PAGE,num_of_links_on_page)):
        if(response.xpath('//li/a/@href')[link].extract().startswith('http') and
            'paradoxwikis' not in response.xpath('//li/a/@href')[link].extract()):
            num_of_valid_links += 1
            outward_links.append(response.xpath('//li/a/@href')[link].extract())
    #print "***"
    #print outward_links
    
    num_of_links = len(outward_links)
    
    tohash = url
    hobject = hashlib.sha256(tohash.encode())
    base_link_hash = str(hobject.hexdigest())
    
    i = 0
    outward_links_hashes = []
    for link in outward_links:
        tohash = link
        hobject = hashlib.sha256(tohash.encode())
        outward_links_hashes.append(str(hobject.hexdigest()))
        i += 1
    
    # if the base_link is already in our dict we move on
    if (base_link_hash not in pItems):
        pItems[base_link_hash] = dict_format(base_link, outward_links, num_of_links,
                                             base_link_hash, outward_links_hashes)
    
        # if a link is new we note it
        print("NEW LINK\t"+
              "Num of Dict Keys: "+str(len(pItems.keys()))+
              "\tNum of Links on Page: "+str(num_of_links_on_page)+
              "\tNum of Links Collected: "+str(num_of_links))
        
        # if a link is new we parse it recursively
        for link in pItems[base_link_hash]['outward_links']:
            num_of_valid_links += 1
            parse_links_beta(link,depth+1)
        
        #for link in range(min(MAX_PER_PAGE,num_of_links_on_page)):
        
            # if the response starts with 'http' its not a short link and thus valid
            # any 'https://paradoxwikis.com/' requests automatically crash the program
            #  so all of them are to be ignored
            #if(response.xpath('//li/a/@href')[link].extract().startswith('http') and
            #   'paradoxwikis' not in response.xpath('//li/a/@href')[link].extract()):
            #    num_of_valid_links += 1
            #    parse_links_beta(response.xpath('//li/a/@href')[link].extract(),depth+1)
    

In [10]:
parse_links_beta('https://www.paradoxplaza.com/',0)
print num_of_valid_links
print pItems

https://www.paradoxplaza.com/
NEW LINK	Num of Dict Keys: 1	Num of Links on Page: 35	Num of Links Collected: 14
	https://www.paradoxplaza.com/account
NEW LINK	Num of Dict Keys: 2	Num of Links on Page: 35	Num of Links Collected: 14
		https://www.paradoxplaza.com/account
		https://www.paradoxplaza.com/register
NEW LINK	Num of Dict Keys: 3	Num of Links on Page: 35	Num of Links Collected: 14
		https://www.paradoxplaza.com/login?original=%2Faccount
NEW LINK	Num of Dict Keys: 4	Num of Links on Page: 35	Num of Links Collected: 14
		https://www.paradoxplaza.com/account
		https://www.paradoxplaza.com/register
		https://www.paradoxplaza.com/browse/
NEW LINK	Num of Dict Keys: 5	Num of Links on Page: 79	Num of Links Collected: 15
		https://www.paradoxplaza.com/new-releases/
NEW LINK	Num of Dict Keys: 6	Num of Links on Page: 47	Num of Links Collected: 15
		https://www.paradoxplaza.com/coming-soon/
NEW LINK	Num of Dict Keys: 7	Num of Links on Page: 57	Num of Links Collected: 15
		https://www.paradoxp

After running the code above the dictionary is populated with all of the link data. Now because python dictionaries are structured identically to JSON files, we will dump all of the collected data to a JSON file.

In [13]:
import json
print pItems
with open('data.json', 'w') as outfile:
    json.dump(pItems, outfile)

{'b3daced18bd70d411e69d6ac5991f126e6edd7a45aa2e4232086cc44ce5b9458': {'outward_links': [u'https://www.paradoxplaza.com/account', u'https://www.paradoxplaza.com/register', u'https://www.paradoxplaza.com/coming-soon/', u'https://www.paradoxplaza.com/account', u'https://www.paradoxplaza.com/register', u'https://www.paradoxplaza.com/coming-soon/?prefn1=pdx-OS-support&prefv1=01', u'https://www.paradoxplaza.com/coming-soon/?prefn1=pdx-OS-support&prefv1=02', u'https://www.paradoxplaza.com/coming-soon/?prefn1=pdx-OS-support&prefv1=03', u'https://www.paradoxplaza.com/coming-soon/?prefn1=pdx-brand&prefv1=Age%20of%20Wonders%3A%20Planetfall', u'https://www.paradoxplaza.com/coming-soon/?prefn1=pdx-brand&prefv1=Battletech', u'https://www.paradoxplaza.com/coming-soon/?prefn1=pdx-brand&prefv1=Crusader%20Kings', u'https://www.paradoxplaza.com/coming-soon/?prefn1=pdx-brand&prefv1=Hearts%20of%20Iron', u'https://www.paradoxplaza.com/coming-soon/?prefn1=pdx-brand&prefv1=Imperator%3A%20Rome', u'https://www.

I highly recommend you open up the json generated from this function. It's quite comprehensive and is about 128kB. While it is rather huge, its very readable and searchable. I reccommend a json prettyfier through (I use the Atom editor with the pretty-json package).

# 7.2 Generating the Stochastic matrix
Creating this matrix should be algorithmically light. Because I am using a dictionary and can only iterate by key, I am going to create a new dict which assosiates the keys with index numbers to make programming the matrix easier.

Edit: I'm kind of losing focus, but cant figure out a way around having a dict for having the index as keys and the hashes as keys without modifying my base dictionary.

In [14]:
#pass hash get index
refHash = {}
#pass index get hash
refIndex = {}
index = 0
for key in pItems:
    refHash[key] = index
    index += 1
    
refIndex = {v: k for k, v in refHash.iteritems()} 
    
print refIndex
print refHash

{0: 'b3daced18bd70d411e69d6ac5991f126e6edd7a45aa2e4232086cc44ce5b9458', 1: '23775afde01959686aea395e860d2ffd2f7248880231d6b457bdf28d4d24ca72', 2: '88e70fdfc0d7351788e058c1c1bd6892e775053d1dc27072949819b23b7b2c1c', 3: '1574bdbe3a0f57032c265a153f15901a8e764e520eb3751df42fac44ebbd89ca', 4: '70bb6dfc807db2f9dde731c68e5888162c58b2240bcf813e5d3610e9ab8906fa', 5: '04d382e5c249a809e079ffbd30f2085f6f192a4a5aec73c25eae00578501fdaf', 6: '268eb2b2f2101106849d05526ad63408a92ed2a0641476fda2f4aae66ec9a9a8', 7: '02edaa08ae2883c05a75bd98bccf82ba0f7387b0b2e0c67e5d2d9e8920bf11e4', 8: '8fc30263883e97fb26cfc52a784cf51914c4089dd8daa87753e874e5d0e56eb5', 9: '9955e458fae57d8ab5df423292e741878e87ca641e68300dd967f722bf9c7f7e', 10: 'a85b6596e173675f06d95cc8c5ac97bc9e2b7349d8458aa97b441c5b061a47fb', 11: 'fbaed0aada9d0509948144722a6747c21e668073de3bfe905dd7822c547fd717', 12: '87e920ba1197aebcd223aed0c3995f8c155bbf9a2b0bc655554759d32cd7c9c8'}
{'b3daced18bd70d411e69d6ac5991f126e6edd7a45aa2e4232086cc44ce5b9458': 0, '

In [101]:
#initializing the stochastic matrix
uniq_links = len(pItems)
sto_matrix = [[0 for col in range(uniq_links)] for row in range(uniq_links)]

#creating print function
def print_sto(extended=False):
    for row in range(uniq_links):
        for col in range(uniq_links):
            print "{:1.4f}".format(sto_matrix[row][col]) if extended else "{:1.2f}".format(sto_matrix[row][col]),
            
        print "" if extended else ' '+pItems[refIndex[row]]['base_link'] 

print_sto()

0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00  https://www.paradoxplaza.com/coming-soon/
0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00  https://www.paradoxplaza.com/login?original=%2Faccount
0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00  https://www.paradoxplaza.com/support-static-info-ca.html
0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00  https://www.paradoxplaza.com/new-releases/
0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00  https://www.paradoxplaza.com/account
0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00  https://www.paradoxplaza.com/browse/
0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00  https://forum.paradoxplaza.com/forum/index.php
0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00  https://www.paradoxplaza.com/
0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00  https://www.paradoxplaza.com/on-sale/
0.00 0.00 0.00 0.00 

Because my dictionary does not count how many times a link is referenced, rather just puts a copy of the URL within the outward_links array, I will calculate each matrix value by dividing the number_of_links by one and adding the weights to the matrix values whenever I come upon their hash.

In [102]:
def calc_sto():
    key_err_cnt = 0
    for row in range(uniq_links):
        key = refIndex[row]
        num_of_links = pItems[key]['num_of_links']
        weight = 0 if num_of_links == 0 else (1.0 / num_of_links)
        
        for links in pItems[key]['outward_links_hashes']:
            try:
                col = refHash[links]
            except KeyError, e:
                #print 'I got a KeyError - reason "%s"' % str(e)
                key_err_cnt += 1
                continue     
            sto_matrix[row][col] += weight
            
    print "Number of Outward Links not accounted for: "+str(key_err_cnt)
    
calc_sto()

Number of Outward Links not accounted for: 62


Now I've realised I did not add unique outward links to my dictionary because I never investigated them with my crawler, they were at the edge of my set depth. Now there are two ways I can fix this.   

1) *I can go back, redo my recusive crawler and have it not stop digging deeper until it hits all known links.  *
This would probubly go terribly and my crawler would get stuck finding every link on the website even if I refined the allowed URLS.  

2) *I can choose to exclude the links that are at the edges of my defined depth.*  
This should be fine as you can easily deduce how many links were not incuded on the stocastic matrix because the rows will no longer add up to one. I will choose this route for now and hope that it doesn't come to bite me when I start diving deeper into HITS and the page rank algorithms.

Lets see what our stochastic matrix looks like now.

In [138]:
print_sto()

0.07 0.00 0.00 0.00 0.13 0.00 0.00 0.00 0.00 0.00 0.13 0.00 0.00  https://www.paradoxplaza.com/coming-soon/
0.07 0.07 0.07 0.07 0.14 0.07 0.07 0.00 0.07 0.07 0.14 0.07 0.07  https://www.paradoxplaza.com/login?original=%2Faccount
0.00 0.00 0.20 0.00 0.40 0.00 0.00 0.00 0.00 0.00 0.40 0.00 0.00  https://www.paradoxplaza.com/support-static-info-ca.html
0.00 0.00 0.00 0.07 0.13 0.00 0.00 0.00 0.00 0.00 0.13 0.00 0.00  https://www.paradoxplaza.com/new-releases/
0.07 0.07 0.07 0.07 0.14 0.07 0.07 0.00 0.07 0.07 0.14 0.07 0.07  https://www.paradoxplaza.com/account
0.00 0.00 0.00 0.00 0.13 0.07 0.00 0.00 0.00 0.00 0.13 0.00 0.00  https://www.paradoxplaza.com/browse/
0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00  https://forum.paradoxplaza.com/forum/index.php
0.07 0.00 0.07 0.07 0.14 0.07 0.07 0.07 0.07 0.07 0.14 0.07 0.07  https://www.paradoxplaza.com/
0.07 0.00 0.07 0.07 0.14 0.07 0.07 0.00 0.14 0.00 0.14 0.00 0.07  https://www.paradoxplaza.com/on-sale/
0.00 0.00 0.00 0.00 

In [190]:
from numpy import *
# fixxed with pretty print
    #change me to change the prcesion displayed
    #float_formatter = lambda x: "%.4f" % x
    #set_printoptions(formatter={'float_kind':float_formatter})

fin_iter = []
 
def pagerank(H):
    n= len(H)
    w = zeros(n)
    rho = 1./n * ones(n);
    for i in range(n):
      if multiply.reduce(H[i]== zeros(n)):
        w[i] = 1
    newH = H + outer((1./n * w),ones(n))
 
    theta=0.85
    G = (theta * newH) + ((1-theta) * outer(1./n * ones(n), ones(n)))
    #by replacing the python default print functions with this one the matricies print nicely with the notebook width set to 100% of the browser width
    print '[',
    for value in rho:
        print "{:1.4f} ".format(value),
    print ']'
    #print rho 
    
    for j in range(10):
        rho = dot(rho,G)
        #more pretty print
        print '[',
        for value in rho:
            print "{:1.4f} ".format(value),
            if(j == 9):
                fin_iter.append(value)
        print ']'
        
        #print rho

In [191]:
pagerank(sto_matrix)

[ 0.0769  0.0769  0.0769  0.0769  0.0769  0.0769  0.0769  0.0769  0.0769  0.0769  0.0769  0.0769  0.0769  ]
[ 0.0537  0.0309  0.0580  0.0537  0.1293  0.0537  0.0496  0.0263  0.0540  0.0403  0.1340  0.0446  0.0493  ]
[ 0.0432  0.0246  0.0475  0.0432  0.1034  0.0432  0.0406  0.0164  0.0434  0.0343  0.1115  0.0368  0.0401  ]
[ 0.0348  0.0199  0.0384  0.0348  0.0835  0.0348  0.0327  0.0131  0.0350  0.0277  0.0902  0.0298  0.0324  ]
[ 0.0281  0.0161  0.0310  0.0281  0.0674  0.0281  0.0265  0.0106  0.0283  0.0224  0.0729  0.0240  0.0262  ]
[ 0.0227  0.0130  0.0251  0.0227  0.0545  0.0227  0.0214  0.0086  0.0229  0.0181  0.0589  0.0194  0.0211  ]
[ 0.0184  0.0105  0.0202  0.0184  0.0440  0.0184  0.0173  0.0069  0.0185  0.0146  0.0476  0.0157  0.0171  ]
[ 0.0148  0.0085  0.0163  0.0148  0.0355  0.0148  0.0139  0.0056  0.0149  0.0118  0.0384  0.0127  0.0138  ]
[ 0.0120  0.0068  0.0132  0.0120  0.0287  0.0120  0.0113  0.0045  0.0121  0.0095  0.0311  0.0102  0.0111  ]
[ 0.0097  0.0055  0.0107  0.

and the top 5 urls after 10 iterations are...

In [206]:
def print_place(places): 
    nums = ['1st: ','2nd: ','3rd: ','4th: ','5th: ']
    for i in range(5):
        print nums[i]+pItems[refIndex[places[i]]]['base_link']

max_val = 0
prev_max = 100000
index = 0
cntr = 0
place_indexes = []

#print fin_iter

for i in range(5):
    for value in fin_iter:
        if value > max_val and value < prev_max:
            max_val = value
            index = cntr
        cntr += 1
        
    place_indexes.append(index)
    prev_max = max_val
    max_val = 0
    cntr = 0

print place_indexes
    
print_place(place_indexes)

[10, 4, 2, 8, 0]
1st: https://www.paradoxplaza.com/register
2nd: https://www.paradoxplaza.com/account
3rd: https://www.paradoxplaza.com/support-static-info-ca.html
4th: https://www.paradoxplaza.com/on-sale/
5th: https://www.paradoxplaza.com/coming-soon/


# 7.4 HITS algorithm
Now time to create a connectivety matrix by modifying our `calc_sto` and `print_sto` functions to suit our needs

In [207]:
#initializing the connectivity matrix
#uniq_links = len(pItems)
conn_matrix = [[0 for col in range(uniq_links)] for row in range(uniq_links)]

#creating print function
def print_conn(extended=False):
    for row in range(uniq_links):
        for col in range(uniq_links):
            print "{:1.4f}".format(conn_matrix[row][col]) if extended else "{:1.2f}".format(conn_matrix[row][col]),
            
        print "" if extended else ' '+pItems[refIndex[row]]['base_link'] 

print_conn()

0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00  https://www.paradoxplaza.com/coming-soon/
0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00  https://www.paradoxplaza.com/login?original=%2Faccount
0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00  https://www.paradoxplaza.com/support-static-info-ca.html
0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00  https://www.paradoxplaza.com/new-releases/
0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00  https://www.paradoxplaza.com/account
0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00  https://www.paradoxplaza.com/browse/
0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00  https://forum.paradoxplaza.com/forum/index.php
0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00  https://www.paradoxplaza.com/
0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00  https://www.paradoxplaza.com/on-sale/
0.00 0.00 0.00 0.00 

In [208]:
def calc_conn():
    key_err_cnt = 0
    for row in range(uniq_links):
        key = refIndex[row]
        num_of_links = pItems[key]['num_of_links']
        weight = 1 #0 if num_of_links == 0 else (1.0 / num_of_links)
        
        for links in pItems[key]['outward_links_hashes']:
            try:
                col = refHash[links]
            except KeyError, e:
                #print 'I got a KeyError - reason "%s"' % str(e)
                key_err_cnt += 1
                continue     
            conn_matrix[row][col] = weight
            
    print "Number of Outward Links not accounted for: "+str(key_err_cnt)
    
calc_conn()

Number of Outward Links not accounted for: 62


In [181]:
print_conn()

1.00 0.00 0.00 0.00 1.00 0.00 0.00 0.00 0.00 0.00 1.00 0.00 0.00  https://www.paradoxplaza.com/coming-soon/
1.00 1.00 1.00 1.00 1.00 1.00 1.00 0.00 1.00 1.00 1.00 1.00 1.00  https://www.paradoxplaza.com/login?original=%2Faccount
0.00 0.00 1.00 0.00 1.00 0.00 0.00 0.00 0.00 0.00 1.00 0.00 0.00  https://www.paradoxplaza.com/support-static-info-ca.html
0.00 0.00 0.00 1.00 1.00 0.00 0.00 0.00 0.00 0.00 1.00 0.00 0.00  https://www.paradoxplaza.com/new-releases/
1.00 1.00 1.00 1.00 1.00 1.00 1.00 0.00 1.00 1.00 1.00 1.00 1.00  https://www.paradoxplaza.com/account
0.00 0.00 0.00 0.00 1.00 1.00 0.00 0.00 0.00 0.00 1.00 0.00 0.00  https://www.paradoxplaza.com/browse/
0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00  https://forum.paradoxplaza.com/forum/index.php
1.00 0.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00  https://www.paradoxplaza.com/
1.00 0.00 1.00 1.00 1.00 1.00 1.00 0.00 1.00 0.00 1.00 0.00 1.00  https://www.paradoxplaza.com/on-sale/
0.00 0.00 0.00 0.00 

Above I've printed our connectivity matrix. Below we will run it through HITS.
I've modified the hits code like the pagerank code to make the print formatting better.

In [209]:
def hits(A):
    n= len(A)
    Au= dot(transpose(A),A)
    Hu = dot(A,transpose(A))
    a = ones(n); h = ones(n)
    
    #more pretty print
    print '[',
    for value in a:
        print "{:1.4f} ".format(value),
    print ']',
    print '[',
    for value in h:
        print "{:1.4f} ".format(value),
    print ']'
    
    for j in range(5):
        a = dot(a,Au)
        a= a/sum(a)
        h = dot(h,Hu)
        h = h/ sum(h)
        
        #more pretty print
        print '[',
        for value in a:
            print "{:1.4f} ".format(value),
        print ']',
        print '[',
        for value in h:
            print "{:1.4f} ".format(value),
        print ']'

In [210]:
hits(conn_matrix)

[ 1.0000  1.0000  1.0000  1.0000  1.0000  1.0000  1.0000  1.0000  1.0000  1.0000  1.0000  1.0000  1.0000  ] [ 1.0000  1.0000  1.0000  1.0000  1.0000  1.0000  1.0000  1.0000  1.0000  1.0000  1.0000  1.0000  1.0000  ]
[ 0.0912  0.0327  0.0803  0.0912  0.1034  0.0912  0.0776  0.0163  0.0871  0.0639  0.1034  0.0748  0.0871  ] [ 0.0503  0.1415  0.0484  0.0503  0.1415  0.0503  0.0000  0.1397  0.1210  0.0000  0.1378  0.1080  0.0112  ]
[ 0.0917  0.0315  0.0798  0.0917  0.1058  0.0917  0.0765  0.0155  0.0869  0.0622  0.1058  0.0739  0.0869  ] [ 0.0457  0.1460  0.0439  0.0457  0.1460  0.0457  0.0000  0.1437  0.1215  0.0000  0.1414  0.1093  0.0112  ]
[ 0.0917  0.0314  0.0798  0.0917  0.1061  0.0917  0.0764  0.0155  0.0869  0.0621  0.1061  0.0738  0.0869  ] [ 0.0452  0.1464  0.0435  0.0452  0.1464  0.0452  0.0000  0.1440  0.1216  0.0000  0.1417  0.1093  0.0113  ]
[ 0.0917  0.0314  0.0798  0.0917  0.1061  0.0917  0.0764  0.0155  0.0869  0.0621  0.1061  0.0738  0.0869  ] [ 0.0452  0.1464  0.0434  0.

# Thoughts

To be honest I am not sure how to read the hits output matrix. Reading the link you provided, they just cut off after showing the HITS alogrithm. And while I would normally be more motivated to continue on because this is interesting, but by now I have spent nearly 20 hours on this assignment, it is Saturday the week after it was due, and I haven't even thouched this weeks homework or my final project. I would have to blame scrapy for the giant time sink this homwork was, but I think it has far more to do with how juypter and scrapy refuse to work well together, as well as the multiple python versions I have installed on my computer.